<a href="https://colab.research.google.com/github/AdamClarkStandke/GenerativeDeepLearning/blob/main/MusicGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Generation

This code comes from David Foster's [notebook](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/11_music/01_transformer/transformer.ipynb) from his book [Generative Deep Learning](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1492041947).

Unlike David Foster's notebook this simple monophonic music generator was trained on the [English Suites](http://www.jsbach.net/midi/midi_english_suites.html) rather than [Suites for Solo Cello](http://www.jsbach.net/midi/midi_solo_cello.html) by [Johann Sebastian Bach](https://en.wikipedia.org/wiki/English_Suites_(Bach). I used to play the piano (thanks to margaret maas the greatest piano teacher ever) and hence that was the reason why I changed the dataset since I do not understand any string type of arraignments 😆

In David's book [Generative Deep Learning](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1492041947) and [repository] (https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/) he details not only this transformer based music generator but also [MuseGAN](https://github.com/salu133445/musegan) which is able to generate polyphonic compositions.

Other than preprocessing  (i.e. converting musical notes and their durations into a computer readable format) and the Sinusoidal positional encoding layer, the model consists of one transformer layer and two output heads of dense form.  

---



In [1]:
import os
import glob
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle as pkl
import keras
import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks
from fractions import Fraction
import music21

In [14]:
def get_midi_note(sample_note, sample_duration):
    new_note = None

    if "TS" in sample_note:
        new_note = music21.meter.TimeSignature(sample_note.split("TS")[0])

    elif "major" in sample_note or "minor" in sample_note:
        tonic, mode = sample_note.split(":")
        new_note = music21.key.Key(tonic, mode)

    elif sample_note == "rest":
        new_note = music21.note.Rest()
        new_note.duration = music21.duration.Duration(
            float(Fraction(sample_duration))
        )
        new_note.storedInstrument = music21.instrument.Violoncello()

    elif "." in sample_note:
        notes_in_chord = sample_note.split(".")
        chord_notes = []
        for current_note in notes_in_chord:
            n = music21.note.Note(current_note)
            n.duration = music21.duration.Duration(
                float(Fraction(sample_duration))
            )
            n.storedInstrument = music21.instrument.Violoncello()
            chord_notes.append(n)
        new_note = music21.chord.Chord(chord_notes)

    elif sample_note == "rest":
        new_note = music21.note.Rest()
        new_note.duration = music21.duration.Duration(
            float(Fraction(sample_duration))
        )
        new_note.storedInstrument = music21.instrument.Violoncello()

    elif sample_note != "START":
        new_note = music21.note.Note(sample_note)
        new_note.duration = music21.duration.Duration(
            float(Fraction(sample_duration))
        )
        new_note.storedInstrument = music21.instrument.Violoncello()

    return new_note

In [2]:
def load_parsed_files_notes(parsed_data_path):
    with open(parsed_data_path, "rb") as f:
        notes = pkl.load(f)
    return notes

def load_parsed_files_duration(parsed_data_path):
    with open(parsed_data_path, "rb") as f:
        durations = pkl.load(f)
    return durations

class SinePositionEncoding(keras.layers.Layer):
    """Sinusoidal positional encoding layer.
    This layer calculates the position encoding as a mix of sine and cosine
    functions with geometrically increasing wavelengths. Defined and formulized
    in [Attention is All You Need](https://arxiv.org/abs/1706.03762).
    Takes as input an embedded token tensor. The input must have shape
    [batch_size, sequence_length, feature_size]. This layer will return a
    positional encoding the same size as the embedded token tensor, which
    can be added directly to the embedded token tensor.
    Args:
        max_wavelength: The maximum angular wavelength of the sine/cosine
            curves, as described in Attention is All You Need. Defaults to
            10000.
    Examples:
    ```python
    # create a simple embedding layer with sinusoidal positional encoding
    seq_len = 100
    vocab_size = 1000
    embedding_dim = 32
    inputs = keras.Input((seq_len,), dtype=tf.float32)
    embedding = keras.layers.Embedding(
        input_dim=vocab_size, output_dim=embedding_dim
    )(inputs)
    positional_encoding = keras_nlp.layers.SinePositionEncoding()(embedding)
    outputs = embedding + positional_encoding
    ```
    References:
     - [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)
    """

    def __init__(
        self,
        max_wavelength=10000,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.max_wavelength = max_wavelength

    def call(self, inputs):
        # TODO(jbischof): replace `hidden_size` with`hidden_dim` for consistency
        # with other layers.
        input_shape = tf.shape(inputs)
        # length of sequence is the second last dimension of the inputs
        seq_length = input_shape[-2]
        hidden_size = input_shape[-1]
        position = tf.cast(tf.range(seq_length), self.compute_dtype)
        min_freq = tf.cast(1 / self.max_wavelength, dtype=self.compute_dtype)
        timescales = tf.pow(
            min_freq,
            tf.cast(2 * (tf.range(hidden_size) // 2), self.compute_dtype)
            / tf.cast(hidden_size, self.compute_dtype),
        )
        angles = tf.expand_dims(position, 1) * tf.expand_dims(timescales, 0)
        # even indices are sine, odd are cosine
        cos_mask = tf.cast(tf.range(hidden_size) % 2, self.compute_dtype)
        sin_mask = 1 - cos_mask
        # embedding shape is [seq_length, hidden_size]
        positional_encodings = (
            tf.sin(angles) * sin_mask + tf.cos(angles) * cos_mask
        )

        return tf.broadcast_to(positional_encodings, input_shape)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_wavelength": self.max_wavelength,
            }
        )
        return config

In [3]:
PARSE_MIDI_FILES = False
PARSED_DATA_PATH = "/content/drive/MyDrive/music/bach/"
DATASET_REPETITIONS = 1

SEQ_LEN = 50
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 12
DROPOUT_RATE = 0.3
FEED_FORWARD_DIM = 256
LOAD_MODEL = False

# optimization
EPOCHS = 200
BATCH_SIZE = 500

GENERATE_LEN = 77

## Creating Dataset

In [4]:
notes = load_parsed_files_notes("/content/drive/MyDrive/music/bach/notes")
durations = load_parsed_files_duration("/content/drive/MyDrive/music/bach/durations")

In [5]:
def create_dataset(elements):
    ds = (
        tf.data.Dataset.from_tensor_slices(elements)
        .batch(BATCH_SIZE, drop_remainder=True)
        .shuffle(1000)
    )
    vectorize_layer = layers.TextVectorization(
        standardize=None, output_mode="int"
    )
    vectorize_layer.adapt(ds)
    vocab = vectorize_layer.get_vocabulary()
    return ds, vectorize_layer, vocab


notes_seq_ds, notes_vectorize_layer, notes_vocab = create_dataset(notes)
durations_seq_ds, durations_vectorize_layer, durations_vocab = create_dataset(
    durations
)
seq_ds = tf.data.Dataset.zip((notes_seq_ds, durations_seq_ds))
notes_vocab_size = len(notes_vocab)
durations_vocab_size = len(durations_vocab)

In [6]:
# Create the training set of sequences and the same sequences shifted by one note
def prepare_inputs(notes, durations):
    notes = tf.expand_dims(notes, -1)
    durations = tf.expand_dims(durations, -1)
    tokenized_notes = notes_vectorize_layer(notes)
    tokenized_durations = durations_vectorize_layer(durations)
    x = (tokenized_notes[:, :-1], tokenized_durations[:, :-1])
    y = (tokenized_notes[:, 1:], tokenized_durations[:, 1:])
    return x, y


ds = seq_ds.map(prepare_inputs).repeat(DATASET_REPETITIONS)

## Building Model

In [7]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

In [8]:
class TransformerBlock(layers.Layer):
    def __init__(
        self,
        num_heads,
        key_dim,
        embed_dim,
        ff_dim,
        name,
        dropout_rate=DROPOUT_RATE,
    ):
        super(TransformerBlock, self).__init__(name=name)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

In [9]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size,
            output_dim=embed_dim,
            embeddings_initializer="he_uniform",
        )
        self.pos_emb = SinePositionEncoding()

    def call(self, x):
        embedding = self.token_emb(x)
        positions = self.pos_emb(embedding)
        return embedding + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [10]:
note_inputs = layers.Input(shape=(None,), dtype=tf.int32)
durations_inputs = layers.Input(shape=(None,), dtype=tf.int32)
note_embeddings = TokenAndPositionEmbedding(
    notes_vocab_size, EMBEDDING_DIM // 2
)(note_inputs)
duration_embeddings = TokenAndPositionEmbedding(
    durations_vocab_size, EMBEDDING_DIM // 2
)(durations_inputs)
embeddings = layers.Concatenate()([note_embeddings, duration_embeddings])
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM, name="attention"
)(embeddings)
note_outputs = layers.Dense(
    notes_vocab_size, activation="softmax", name="note_outputs"
)(x)
duration_outputs = layers.Dense(
    durations_vocab_size, activation="softmax", name="duration_outputs"
)(x)
model = models.Model(
    inputs=[note_inputs, durations_inputs],
    outputs=[note_outputs, duration_outputs],  # attention_scores
)
model.compile(
    "adam",
    loss=[
        losses.SparseCategoricalCrossentropy(),
        losses.SparseCategoricalCrossentropy(),
    ],
)
att_model = models.Model(
    inputs=[note_inputs, durations_inputs], outputs=attention_scores
)

In [11]:
# Create a MusicGenerator checkpoint
class MusicGenerator(callbacks.Callback):
    def __init__(self, index_to_note, index_to_duration, top_k=10):
        self.index_to_note = index_to_note
        self.note_to_index = {
            note: index for index, note in enumerate(index_to_note)
        }
        self.index_to_duration = index_to_duration
        self.duration_to_index = {
            duration: index for index, duration in enumerate(index_to_duration)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def get_note(self, notes, durations, temperature):
        sample_note_idx = 1
        while sample_note_idx == 1:
            sample_note_idx, note_probs = self.sample_from(
                notes[0][-1], temperature
            )
            sample_note = self.index_to_note[sample_note_idx]

        sample_duration_idx = 1
        while sample_duration_idx == 1:
            sample_duration_idx, duration_probs = self.sample_from(
                durations[0][-1], temperature
            )
            sample_duration = self.index_to_duration[sample_duration_idx]

        new_note = get_midi_note(sample_note, sample_duration)

        return (
            new_note,
            sample_note_idx,
            sample_note,
            note_probs,
            sample_duration_idx,
            sample_duration,
            duration_probs,
        )

    def generate(self, start_notes, start_durations, max_tokens, temperature):
        attention_model = models.Model(
            inputs=self.model.input,
            outputs=self.model.get_layer("attention").output,
        )

        start_note_tokens = [self.note_to_index.get(x, 1) for x in start_notes]
        start_duration_tokens = [
            self.duration_to_index.get(x, 1) for x in start_durations
        ]
        sample_note = None
        sample_duration = None
        info = []
        midi_stream = music21.stream.Stream()

        midi_stream.append(music21.clef.BassClef())

        for sample_note, sample_duration in zip(start_notes, start_durations):
            new_note = get_midi_note(sample_note, sample_duration)
            if new_note is not None:
                midi_stream.append(new_note)

        while len(start_note_tokens) < max_tokens:
            x1 = np.array([start_note_tokens])
            x2 = np.array([start_duration_tokens])
            notes, durations = self.model.predict([x1, x2], verbose=0)

            repeat = True

            while repeat:
                (
                    new_note,
                    sample_note_idx,
                    sample_note,
                    note_probs,
                    sample_duration_idx,
                    sample_duration,
                    duration_probs,
                ) = self.get_note(notes, durations, temperature)

                if (
                    isinstance(new_note, music21.chord.Chord)
                    or isinstance(new_note, music21.note.Note)
                    or isinstance(new_note, music21.note.Rest)
                ) and sample_duration == "0.0":
                    repeat = True
                else:
                    repeat = False

            if new_note is not None:
                midi_stream.append(new_note)

            _, att = attention_model.predict([x1, x2], verbose=0)

            info.append(
                {
                    "prompt": [start_notes.copy(), start_durations.copy()],
                    "midi": midi_stream,
                    "chosen_note": (sample_note, sample_duration),
                    "note_probs": note_probs,
                    "duration_probs": duration_probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_note_tokens.append(sample_note_idx)
            start_duration_tokens.append(sample_duration_idx)
            start_notes.append(sample_note)
            start_durations.append(sample_duration)

            if sample_note == "START":
                break

        return info

## Training Model

In [12]:
# Tokenize starting prompt
music_generator = MusicGenerator(notes_vocab, durations_vocab)

model.fit(
    ds,
    epochs=EPOCHS,
    callbacks=[
        music_generator,
    ],
)

# Save the final model
model.save("/content/drive/MyDrive/music/Transformer")

Epoch 1/200
 6/88 [=>............................] - ETA: 44s - loss: 6.6930 - note_outputs_loss: 4.3098 - duration_outputs_loss: 2.3832

88/88 [==============================] - 24s 192ms/step - loss: 4.8459 - note_outputs_loss: 3.5727 - duration_outputs_loss: 1.2732
Epoch 2/200
88/88 [==============================] - 14s 154ms/step - loss: 4.3706 - note_outputs_loss: 3.3182 - duration_outputs_loss: 1.0524
Epoch 3/200
88/88 [==============================] - 11s 128ms/step - loss: 3.7023 - note_outputs_loss: 2.8127 - duration_outputs_loss: 0.8897
Epoch 4/200
88/88 [==============================] - 14s 163ms/step - loss: 3.4722 - note_outputs_loss: 2.6201 - duration_outputs_loss: 0.8521
Epoch 5/200
88/88 [==============================] - 11s 128ms/step - loss: 3.3426 - note_outputs_loss: 2.5343 - duration_outputs_loss: 0.8084
Epoch 6/200
88/88 [==============================] - 12s 137ms/step - loss: 3.2710 - note_outputs_loss: 2.4787 - duration_outputs_loss: 0.7923
Epoch 7/200
88/88 [==============================] - 14s 158ms/step - loss: 3.2248 - note_outputs_loss: 2.4377 - duration_outputs_loss: 0.7871
Epoch 8/200

## Generating Music

In [15]:
#generating music with temperature 0.5
info = music_generator.generate(
    ["START"], ["0.0"], max_tokens=50, temperature=0.5
)
midi_stream = info[-1]["midi"].chordify()

In [17]:
midi_stream.show('text')

{0.0} <music21.clef.BassClef>
{0.0} <music21.key.Key of C major>
{0.0} <music21.meter.TimeSignature 1/8>
{0.0} <music21.chord.Chord A5>
{0.5} <music21.meter.TimeSignature 3/2>
{0.5} <music21.chord.Chord A5>
{1.0} <music21.chord.Chord A5>
{1.5} <music21.chord.Chord A5>
{2.0} <music21.chord.Chord A5>
{2.5} <music21.chord.Chord E5>
{3.0} <music21.chord.Chord F#5>
{3.5} <music21.chord.Chord F#5>
{4.0} <music21.chord.Chord E-5>
{4.5} <music21.chord.Chord E5>
{5.0} <music21.chord.Chord F#5>
{5.5} <music21.chord.Chord G5>
{6.0} <music21.chord.Chord A5>
{6.5} <music21.chord.Chord B5>
{7.0} <music21.chord.Chord C6>
{7.5} <music21.chord.Chord C6>
{8.0} <music21.chord.Chord B5>
{8.5} <music21.chord.Chord A5>
{9.0} <music21.chord.Chord G#5>
{9.5} <music21.chord.Chord F#5>
{10.0} <music21.chord.Chord G#5>
{10.5} <music21.chord.Chord B5>
{11.0} <music21.chord.Chord A5>
{11.5} <music21.chord.Chord G#5>
{12.0} <music21.chord.Chord A5>
{12.5} <music21.chord.Chord C6>
{13.0} <music21.chord.Chord G#5>
{1

In [21]:
# generating music with temperature 1
info = music_generator.generate(
    ["START"], ["0.0"], max_tokens=50, temperature=1
)
midi_stream_two = info[-1]["midi"].chordify()
midi_stream_two.show('text')

{0.0} <music21.clef.BassClef>
{0.0} <music21.meter.TimeSignature 3/4>
{0.0} <music21.chord.Chord D5>
{0.25} <music21.chord.Chord E-5>
{0.5} <music21.chord.Chord E-5>
{1.0} <music21.note.Rest eighth>
{1.5} <music21.chord.Chord D5>
{2.0} <music21.chord.Chord C5>
{2.5} <music21.chord.Chord C5>
{2.75} <music21.chord.Chord D5>
{3.0} <music21.note.Rest eighth>
{3.5} <music21.chord.Chord G4>
{4.0} <music21.chord.Chord A4>
{4.5} <music21.chord.Chord A4>
{5.0} <music21.chord.Chord A4>
{5.25} <music21.chord.Chord B-4>
{5.5} <music21.chord.Chord B-4>
{6.0} <music21.chord.Chord C5>
{6.5} <music21.chord.Chord C5>
{7.0} <music21.chord.Chord C5>
{7.5} <music21.chord.Chord A4>
{7.75} <music21.chord.Chord A4>
{8.0} <music21.chord.Chord F4>
{8.5} <music21.chord.Chord C5>
{9.0} <music21.chord.Chord B-4>
{9.25} <music21.chord.Chord A4>
{9.5} <music21.chord.Chord G4>
{10.0} <music21.chord.Chord C5>
{10.5} <music21.chord.Chord C5>
{11.0} <music21.chord.Chord B-4>
{11.5} <music21.chord.Chord B-4>
{12.0} <mus

In [29]:
#generating music with temperature 0.42
info = music_generator.generate(
    ["START"], ["0.0"], max_tokens=50, temperature=0.42
)
midi_stream_three = info[-1]["midi"].chordify()
midi_stream_three.show('text')

{0.0} <music21.clef.BassClef>
{0.0} <music21.key.Key of C major>
{0.0} <music21.meter.TimeSignature 1/8>
{0.0} <music21.chord.Chord A4>
{0.5} <music21.meter.TimeSignature 3/2>
{0.5} <music21.chord.Chord A4>
{0.75} <music21.chord.Chord A4>
{1.0} <music21.chord.Chord A4>
{1.25} <music21.chord.Chord A4>
{1.5} <music21.chord.Chord A4>
{1.75} <music21.chord.Chord A4>
{2.0} <music21.chord.Chord E4>
{2.25} <music21.chord.Chord C#3>
{2.5} <music21.chord.Chord D3>
{2.75} <music21.chord.Chord E4>
{3.0} <music21.note.Rest 16th>
{3.25} <music21.chord.Chord F#4>
{3.5} <music21.chord.Chord F#4>
{3.75} <music21.chord.Chord F#4>
{4.0} <music21.chord.Chord F#4>
{4.25} <music21.chord.Chord G4>
{4.5} <music21.chord.Chord F#4>
{4.75} <music21.chord.Chord D4>
{5.0} <music21.chord.Chord E4>
{5.25} <music21.chord.Chord F4>
{5.5} <music21.chord.Chord G4>
{5.75} <music21.chord.Chord B-4>
{6.0} <music21.chord.Chord A4>
{6.25} <music21.chord.Chord G4>
{6.5} <music21.chord.Chord C5>
{6.75} <music21.chord.Chord C5

In [32]:
#generating music with temperature 1 and 100 tokens for fun
info = music_generator.generate(
    ["START"], ["0.0"], max_tokens=100, temperature=1
)
midi_stream_four = info[-1]["midi"].chordify()
midi_stream_four.show('text')

{0.0} <music21.clef.BassClef>
{0.0} <music21.key.Key of C major>
{0.0} <music21.meter.TimeSignature 1/4>
{0.0} <music21.chord.Chord E4>
{0.3333} <music21.note.Rest 2/3ql>
{1.0} <music21.meter.TimeSignature 4/4>
{1.0} <music21.chord.Chord G5>
{1.5} <music21.chord.Chord G5>
{2.0} <music21.chord.Chord B2>
{2.25} <music21.chord.Chord C3>
{2.5} <music21.chord.Chord G4>
{4.0} <music21.chord.Chord A4>
{4.1667} <music21.chord.Chord A4>
{4.25} <music21.chord.Chord B2>
{4.5} <music21.chord.Chord G5>
{4.75} <music21.chord.Chord B-4>
{5.0} <music21.chord.Chord A5>
{5.25} <music21.chord.Chord F#5>
{5.75} <music21.chord.Chord G5>
{6.0} <music21.chord.Chord G5>
{6.25} <music21.chord.Chord G5>
{7.5} <music21.chord.Chord G5>
{7.75} <music21.chord.Chord G5>
{7.9167} <music21.chord.Chord C5>
{8.0} <music21.chord.Chord D5>
{8.5} <music21.chord.Chord F#5>
{9.25} <music21.chord.Chord G5>
{9.5} <music21.chord.Chord G5>
{10.0} <music21.chord.Chord G5>
{10.1667} <music21.chord.Chord D5>
{10.5} <music21.chord.C